In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ete3

import os
from tqdm import tqdm

from src.phylogenetic_signal import PagelsLambda
from src.ihmp import get_diffs
from src.greengenes import (
    parse_similarity_map, merge_otu_table, combine_similarity_maps
)


In [2]:
%%bash

# Download greengenes OTUs dataset, if necessary
if ! [ -d greengenes/data/gg_13_5_otus/ ]; then
    if ! [ -f greengenes/data/gg_13_5_otus.tar.gz ]; then
        wget -O greengenes/data/gg_13_5_otus.tar.gz \
            https://gg-sg-web.s3-us-west-2.amazonaws.com/downloads/greengenes_database/gg_13_5/gg_13_5_otus.tar.gz
    fi
    tar -xvf greengenes/data/gg_13_5_otus.tar.gz -C greengenes/data/
fi

In [3]:
# I load OTU tables using the basic iHMP scripts from earlier notebooks

data = {
    name: get_diffs(name, get_abundances=True, log=False) # undo log transform
    for name in ["ibd", "moms", "t2d"]
}


In [4]:
# Sanity check 1: relative abundances continue to sum to 1
# They do!!!
similarity_map99 = parse_similarity_map(99)
similarity_map = similarity_map99
for x in [97, 94, 91, 88, 85, 82, 79, 76, 73, 70, 67, 64, 61]:
    similarity_map = combine_similarity_maps(
        [similarity_map, parse_similarity_map(x)]
    )
    clustered = merge_otu_table(data["ibd"], similarity_map)
    s = clustered.sum(axis=1)
    print(f"{x} {clustered.shape} {s.min():.3f} {s.mean():.3f} {s.max():.3f}")

/home/phil/phylosig/src/greengenes.py:207: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_otu_table[cluster_id] = otu_table[existing_otus].sum(axis=1)


97 (98, 1370) 1.000 1.000 1.000
94 (98, 1171) 1.000 1.000 1.000
91 (98, 799) 1.000 1.000 1.000
88 (98, 536) 1.000 1.000 1.000
85 (98, 337) 1.000 1.000 1.000
82 (98, 203) 1.000 1.000 1.000
79 (98, 118) 1.000 1.000 1.000
76 (98, 82) 1.000 1.000 1.000
73 (98, 43) 1.000 1.000 1.000
70 (98, 24) 1.000 1.000 1.000
67 (98, 13) 1.000 1.000 1.000
64 (98, 8) 1.000 1.000 1.000
61 (98, 6) 1.000 1.000 1.000


In [7]:
# Get lambdas for each cutoff

# for name in names:
#     outpath = f"./results/{name}_{ab_str}pls_top{TOP_N}.tsv"
#     print(outpath)
#     if os.path.exists(outpath):
#         print(f"Skipping {outpath} because it already exists.")
#         continue
#     pls, tree = pagels_dataframe(data[name], GG_TREE)
#     pls.to_csv(outpath, sep="\t")
#     tree.write(
#         format=1, outfile=f"./results/{name}_{ab_str}tree_top{TOP_N}.nwk"
#     )


lambdas = []

similarity_map99 = parse_similarity_map(99)
similarity_map = similarity_map99
for x in [99, 97, 94, 91, 88, 85, 82, 79, 76, 73, 70, 67, 64, 61]:

    # Non-redundancy check
    outpath = f"./results/ibd_pls_cutoff{x}.tsv"
    if os.path.exists(outpath):
        print(f"Skipping {outpath} because it already exists.")
        continue

    lambdas_cutoff = []

    # Get GreenGenes data
    if x == 99:
        similarity_map = similarity_map99
    else:
        similarity_map = combine_similarity_maps(
            [similarity_map, parse_similarity_map(x)]
        )
    tree = ete3.Tree(
        f"/home/phil/phylosig/greengenes/data/gg_13_5_otus/trees/{x}_otus.tree",
        format=1,
        quoted_node_names=True,
    )

    # Filter OTU table
    clustered = merge_otu_table(data["ibd"], similarity_map)
    s = clustered.sum(axis=1)
    print(f"{x} {clustered.shape} {s.min():.3f} {s.mean():.3f} {s.max():.3f}")

    # Filter tree, init PL object
    tree.prune(clustered.columns)
    pl = PagelsLambda(tree)

    # Get lambdas
    for sample in tqdm(clustered.index):
        pl.fit(clustered.loc[sample])
        lambdas_cutoff.append({"sample": sample, "lambda": pl.lam})
    
    lambdas_cutoff_df = pd.DataFrame(lambdas_cutoff)
    lambdas_cutoff_df.to_table(outpath)

    lambdas_cutoff_df["cutoff"] = x
    lambdas.append(lambdas_cutoff_df)

lambdas_df = pd.concat(lambdas)

Skipping ./results/ibd_pls_cutoff99.tsv because it already exists.


/home/phil/phylosig/src/greengenes.py:207: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_otu_table[cluster_id] = otu_table[existing_otus].sum(axis=1)


97 (98, 1370) 1.000 1.000 1.000


In [ ]:
import seaborn as sns

cluster_lambda = pd.DataFrame(columns=["sample", "lambda", "cutoff"])
for x in [99, 97, 94, 91, 88, 85, 82, 79, 76, 73, 70, 67, 64, 61]:
    result = pd.read_table(f"./results/ibd_pls_cutoff{x}.tsv")
    result["cutoff"] = x
    cluster_lambda = cluster_lambda.append(result)

# Violin plot for each cutoff
fig, ax = plt.subplots(figsize=(10, 6))
ax = sns.violinplot(
    x="cutoff", y="lambda", data=cluster_lambda, ax=ax, inner="quartile"
)